### Contributions from:

Novin Shahroudi - surprise-me-2-with-lstm

https://www.kaggle.com/novinsh/surprise-me-2-with-lstm

This is a kernel improved from above kernel. 
What I tried here is 
1. to add new flag about before holiday
2. to delete columns whose air_store_id  is in train data but  not in test data.


# Overview
## 1. EDA
- 最大来客数
- 祝前日

## 2.  feature engineering
- testにないstoreをtrainから除外
- 祝前日の追加 (train, testの両方にカラムを追加する)

## 3. fit & predict

## 1. EDA
いろいろとplotしてみる
- 最大来客数
- 祝前日が関係するか

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
data = {
    'tra':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_visit_data.csv'),
    'as':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_store_info.csv'),
    'hs':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_store_info.csv'),
    'ar':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_reserve.csv'),
    'hr':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_reserve.csv'),
    'id':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/store_id_relation.csv'),
    'tes':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/sample_submission.csv'),
    'hol':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/date_info.csv').rename(columns={
        'calendar_date': 'visit_date'
    })
}

## 1 - 1. 最大来客数の分布

In [ ]:
plt.hist(data['tra'].groupby('air_store_id').max()['visitors'], bins=200)
plt.xlabel('max visitors', size=15)
plt.show()

In [ ]:
over100 = data['tra'][data['tra']['visitors'] > 100]
print(len(over100['air_store_id'].unique()), ': Num of stores which can take in more than 100 people.')

We find:
- ほとんどの店は50~150人程度
- ただし100~800人程度まで収容する店が150店舗ほどあるので、max_visitorをtraiing dataに追加したほうが良さげ。

## 1-2. 祝前日と来客数の相関

In [ ]:
# 祝前日フラグの追加
data['hol']['before_holiday_flg'] = data['hol'].shift(-1)['holiday_flg']
data['hol'].ix[data['hol'].index[-1], 'before_holiday_flg'] = 1

# visitorデータと結合
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
# 祝前日orそれ以外の日における、来客数の平均を出す histグラム？
# 相関がありそうなら、モデル作成時に使用する

In [ ]:
train.head()

In [ ]:
sum_visitor = train.groupby('before_holiday_flg').sum()['visitors']
sum_date = train.groupby('before_holiday_flg').count()['visit_date']
sum_visitor[0]/sum_date[0], sum_visitor[1]/sum_date[1]
plt.bar(['not before holiday', 'before holiday'], np.array([sum_visitor[0]/sum_date[0], sum_visitor[1]/sum_date[1]]))
plt.title('Average of visitors', size=15)
plt.ylabel('num of visitors')
plt.show()

We find:
- 祝前日はそれ以外の日と比べて、1.2倍ほどの来客数があるので、特徴量データに加えて良いかも。

# 2. feature engineering / 3. fit & predict

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy.ndimage.interpolation import shift
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv2D, MaxPooling2D
from math import sqrt, floor
from matplotlib import pyplot as plt

data = {
    'tra':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_visit_data.csv'),
    'as':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_store_info.csv'),
    'hs':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_store_info.csv'),
    'ar':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/air_reserve.csv'),
    'hr':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/hpg_reserve.csv'),
    'id':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/store_id_relation.csv'),
    'tes':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/sample_submission.csv'),
    'hol':
    pd.read_csv('../input/recruit-restaurant-visitor-forecasting/date_info.csv').rename(columns={
        'calendar_date': 'visit_date'
    })
}

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

# 1. 祝前日フラグの追加
data['hol']['before_holiday_flg'] = data['hol'].shift(-1)['holiday_flg']
data['hol'].ix[data['hol'].index[-1], 'before_holiday_flg'] = 1

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)
#OPTIMIZED BY JEROME VALLET
tmp = data['tra'].groupby(['air_store_id','dow']).agg({'visitors' : [np.min,np.mean,np.median,np.max,np.size]}).reset_index()
tmp.columns = ['air_store_id', 'dow', 'min_visitors', 'mean_visitors', 'median_visitors','max_visitors','count_observations']
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date

# 2. テストにないair_atore_idを落とす
test_store_id = data['tes']['air_store_id'].unique()
tra_store_id = data['tra']['air_store_id'].unique()
not_in_test = list()
for tra in tra_store_id:
    if tra not in test_store_id:
        not_in_test.append(tra)
not_in_test_index = data['tra'][data['tra'].air_store_id.isin(not_in_test)].index
data['tra'] = data['tra'].drop(not_in_test_index)

train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

# train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

x_train = train.drop(['air_store_id','visit_date','visitors'], axis=1)
y_train = np.log1p(train['visitors'].values)
print("max visits: ", np.max(train['visitors'].values))
print("min visits: ", np.min(train['visitors'].values))


y_test = test['visitors'].as_matrix()
x_test = test.drop(['id','air_store_id','visit_date','visitors'], axis=1)


#### My Contribution Begins from here (which is only the model) ####

# Define the scaler 
scaler = StandardScaler().fit(x_train)
# scaler = MinMaxScaler().fit(x_train)


# Scale the train set
x_train = scaler.transform(x_train)

# Scale the test set
x_test = scaler.transform(x_test)
    
# Set random seed
np.random.seed(7)

print("--- shape report ---")
print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_test: ", x_test.shape)

# split the training for validation
rate = 1.0 
train_sample_size = floor(x_train.shape[0]*rate)
# commented out the validation
# x_valid = np.copy(x_train[train_sample_size:,:])
# y_valid = np.copy(y_train[train_sample_size:])
x_train = x_train[:train_sample_size,:]
y_train = y_train[:train_sample_size]

x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
# x_valid = x_valid.reshape((x_valid.shape[0], 1, x_valid.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

print("-- network input --")
print("X_train: ", x_train.shape)
print("y_train: ", y_train.shape)
# print("X_valid: ", x_valid.shape)
# print("y_valid: ", y_valid.shape)
print("X_test: ", x_test.shape)


# design network
model = Sequential()
model.add(LSTM(100, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

# fit network
# history = model.fit(x_train, y_train, epochs=10, batch_size=1000, \
        # validation_data=(x_valid, y_valid), verbose=2, shuffle=False)
history = model.fit(x_train, y_train, epochs=600, batch_size=500, \
                    verbose=2, shuffle=False)

# plot history
plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()
 
# make a prediction for test data
yhat = model.predict(x_test)
# yhat = model.predict(x_valid)
# yhat = model.predict(x_train)
visitors = np.absolute(np.expm1(yhat))
test['visitors'] = visitors 
test[['id','visitors']].to_csv('submission_addflag.csv', index=False, float_format='%.3f')

# calculate RMSE for the validation set
# rmse = sqrt(mean_squared_error(yhat, y_valid))
# rmse = sqrt(mean_squared_error(yhat, y_train))
# print('Test RMSE: %.3f' % rmse)